In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree

### Initiate webdriver

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

### Getting page links

In [ ]:
page_links = []
for page in range(1,23): 
    url = 'https://toom.de/c/bauen-renovieren/holz?page={}'.format(page)
    browser.get(url)
    
    browser.implicitly_wait(15)
    actions = ActionChains(browser)
    for _ in range(25):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        sleep(0.1)
    
    sleep(20)       
    elems = browser.find_elements_by_xpath("//a[@href]")
    links_per_page = []
    for elem in elems:
        if '/p/' in elem.get_attribute("href") and elem.get_attribute("href") not in page_links:
            page_links.append(elem.get_attribute("href"))
            links_per_page.append(elem.get_attribute("href"))
    print('Page', page, 'Links', len(links_per_page))

In [ ]:
len(page_links)

In [ ]:
url = 'https://toom.de/c/garten-freizeit/gartenbau-ausstattung/gartenbaustoffe/terrassenhoelzer?filterMaterial=Holz'
browser.get(url)

In [ ]:
elems = browser.find_elements_by_xpath("//a[@href]")
for elem in elems:
    if '/p/' in elem.get_attribute("href") and elem.get_attribute("href") not in page_links:
        page_links.append(elem.get_attribute("href"))
len(page_links)

In [ ]:
#with open('Toom-Wood-Links.pkl', 'wb') as f:
    #pickle.dump(page_links, f)

In [ ]:
#with open('Toom-Wood-Links.pkl', 'rb') as f:
    #page_links = pickle.load(f)

## Selecting options. Getting links with options

In [ ]:
s = cfscrape.create_scraper()
scraper = cfscrape.create_scraper(sess=s)

In [ ]:
%%time
check = ['Variante auswählen',': ',', ',':']
#links_options = []
for i in range(len(page_links)):
    link = page_links[i]
    sleep(0.5)
    if i%100==0:
        print(i)
        sleep(randint(1,5))
    #browser.get(link)
    scraped_html = scraper.get(link).content
    html = etree.HTML(scraped_html)

    # Checking mult options
    raw_options = html.xpath('//*[@class="o-product__variants l-col l-col--medium-12"]//text()')
    
    options=[]
    for opt in raw_options:
        if all([opt !=x for x in check]) and len(re.findall(r'\d',opt))==0:
            options.append(opt)
    if options:
        links_options.append(link)
        print(i, options, link)

In [ ]:
#with open('Toom-Wood-Options_Links.pkl', 'wb') as f:
    #pickle.dump(links_options, f)

## Getting Plain Pages

In [ ]:
plain_pages = [x for x in page_links if x not in links_options]
len(plain_pages)

In [ ]:
df_products = pd.DataFrame([])
except_links = []

In [ ]:
%%time
for i, url in enumerate(plain_pages[:15]):
    if i%5==0:
        print(i, 'Products Table:', df_products.shape)
        
        #with open('Toom-Wood-Products.pkl', 'wb') as f:
            #pickle.dump(df_products, f)
    
    browser.get(url)
    sleep(randint(2,4))
    try:
    
        #Header
        path = '/html/body/div[1]/div/div[2]/section[1]/div/div[2]/div[2]/div/div/h1'
        header = browser.find_element_by_xpath(path).text

        # Price
        path = '//*[@class="m-price-box__value m-price-box--highlight-price"]'
        price = browser.find_element_by_xpath(path).text

        # SKU
        path = '//*[@class="a-paragraph a-paragraph--pico a-paragraph--grey-dark"]'
        sku = browser.find_element_by_xpath(path).text


        # Delivery
        # clicking on delivery
        path = '//*[@class="m-buybox-tabs__title"]'
        browser.find_element_by_xpath(path).click()

        # delivery status
        path = '//*[@class="o-tabbed-buybox__price-and-text"]'
        delivert_price = browser.find_element_by_xpath(path).text

        # Description

        path = '//*[@class="a-list l-space-top-s l-space-bottom-s"]'
        first_description = browser.find_element_by_xpath(path).text.split('\n')
        
        path = '//*[@class="a-paragraph l-space-top-s l-space-bottom-s"]'
        second_description = browser.find_element_by_xpath(path).text.split('\n')
        
        description = '; '.join(first_description) + ' | '+ '; '.join(second_description)

        # Main Table
        # Checking More Option
        scraped_html = scraper.get(url).content
        html = etree.HTML(scraped_html)
        more = html.xpath('//*[@class="a-link m-table__show-more"]//text()')
        if more:
            # Click more to show more data
            path = '//*[@class="a-link m-table__show-more"]'
            browser.find_element_by_xpath(path).click()
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.findAll("table", { "class" : "m-table__content l-space-top-m" })
        else:
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.findAll("table", { "class" : "m-table__content m-table--minimized l-space-top-m" })

        # Getting Main Table
        main_table_data = {}
        for i in range(1,20):
            try:
                key = table[0].find_all('tr')[i].find_all('th')[0].text
                value = table[0].find_all('tr')[i].find_all('td')[0].text
                main_table_data[key] = value
            except:
                pass

        main_table = pd.DataFrame(main_table_data, index=[0])
        main_table['Header'] = header
        main_table['Price'] = price
        main_table['SKU'] = sku
        main_table['Delivery-Price'] = delivert_price
        main_table['Description'] = description
        main_table['URL'] = url

        # Adding data to the main Table
        df_products = pd.concat([df_products, main_table], ignore_index=True)
    except:
        except_links.append(url)
        print(url)

In [ ]:
#with open('Toom-Wood-Products.pkl', 'wb') as f:
    #pickle.dump(df_products, f)

In [ ]:
df_products.shape

## Getting pages with selecting Options

### Function to get page data

In [ ]:
def get_page_date(url):
    #Header
    path = '/html/body/div[1]/div/div[2]/section[1]/div/div[2]/div[2]/div/div/h1'
    header = browser.find_element_by_xpath(path).text

    # Price
    path = '//*[@class="m-price-box__value m-price-box--highlight-price"]'
    price = browser.find_element_by_xpath(path).text

    # SKU
    path = '//*[@class="a-paragraph a-paragraph--pico a-paragraph--grey-dark"]'
    sku = browser.find_element_by_xpath(path).text


    # Delivery
    # clicking on delivery
    path = '//*[@class="m-buybox-tabs__title"]'
    browser.find_element_by_xpath(path).click()

    # delivery status
    path = '//*[@class="o-tabbed-buybox__price-and-text"]'
    delivert_price = browser.find_element_by_xpath(path).text

    # Description

    path = '//*[@class="a-list l-space-top-s l-space-bottom-s"]'
    description = browser.find_element_by_xpath(path).text.split('\n')
    description = '; '.join(description)

    # Main Table
    # Checking More Option
    scraped_html = scraper.get(url).content
    html = etree.HTML(scraped_html)
    more = html.xpath('//*[@class="a-link m-table__show-more"]//text()')
    if more:
        # Click more to show more data
        path = '//*[@class="a-link m-table__show-more"]'
        browser.find_element_by_xpath(path).click()
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.findAll("table", { "class" : "m-table__content l-space-top-m" })
    else:
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.findAll("table", { "class" : "m-table__content m-table--minimized l-space-top-m" })

    # Getting Main Table
    main_table_data = {}
    for i in range(1,20):
        try:
            key = table[0].find_all('tr')[i].find_all('th')[0].text
            value = table[0].find_all('tr')[i].find_all('td')[0].text
            main_table_data[key] = value
        except:
            pass

    main_table = pd.DataFrame(main_table_data, index=[0])
    main_table['Header'] = header
    main_table['Price'] = price
    main_table['SKU'] = sku
    main_table['Delivery-Price'] = delivert_price
    main_table['Description'] = description
    main_table['URL'] = browser.current_url

    return main_table

### Getting data

In [ ]:
%%time
start = 0
for i in range(start, 4):
    url = links_options[i]
    
    if url in df_products['URL'].unique() and i!=start:
        continue
    else:
        pass
    
    sleep(1)
    scraped_html = scraper.get(url).content
    html = etree.HTML(scraped_html)

    # Checking mult options
    raw_options = html.xpath('//*[@class="o-product__variants l-col l-col--medium-12"]//text()')
    
    options=[]
    check = ['Variante auswählen',': ',', ',':']
    for opt in raw_options:
        if all([opt !=x for x in check]) and len(re.findall(r'\d',opt))==0:
            options.append(opt)
    
    # Option 1
    if len(options)==1:
        print(i, 'Plain Option', options)
        
        # Getting Options inside the page
        browser.get(url)
        sleep(4)
        attr_str = browser.find_element_by_xpath('//*[@class="m-variants__attribute"]').get_attribute('innerHTML')
        num_tags = len(re.findall('a-tag m-variants__attribute-tag', attr_str))
        
        # Looping through the options
        for opt in range(2,2+num_tags):
            sleep(6)
            path ='/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[2]/div/div/div[{}]/label'.format(opt)
            browser.find_element_by_xpath(path).click()
            
            # Scrolling page
            actions = ActionChains(browser)
            for _ in range(15):
                actions.send_keys(Keys.PAGE_DOWN).perform()
                sleep(0.1)
            sleep(0.2)
            for _ in range(15):
                actions.send_keys(Keys.PAGE_UP).perform()
                sleep(0.1)
            
            sleep(7)
            main_table = get_page_date(url)
            #display(main_table)
            
            
            # Adding data to the main Table
            df_products = pd.concat([df_products, main_table], ignore_index=True)
            print(df_products.shape)

    
    # Option 2
    elif len(options)==2 and options[0]==options[1]:
        print(i, 'Plain Option with dropdown', options)
        
        # Get page
        browser.get(url)
        arrow_path ='//*[@class="a-icon a-icon--arrow-down m-variants__selection-icon m-variants__selection-icon--closed"]'

        attr_str = browser.find_element_by_xpath('//*[@class="m-variants__attribute"]').get_attribute('innerHTML')
        num_tags = len(re.findall('a-tag m-variants__attribute-tag', attr_str))
        
        
        for i in range(2,2+num_tags):
            sleep(3)
            browser.find_element_by_xpath(arrow_path).click()
            sleep(2)

            path = '/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[3]/div/div/div[{}]/label'.format(i)
            browser.find_element_by_xpath(path).click()
            
            # Scrolling page
            actions = ActionChains(browser)
            for _ in range(15):
                actions.send_keys(Keys.PAGE_DOWN).perform()
                sleep(0.1)
            sleep(0.2)
            for _ in range(10):
                actions.send_keys(Keys.PAGE_UP).perform()
                sleep(0.1)
                    
            sleep(5)
            main_table = get_page_date(url)
            #display(main_table)
            
            
            # Adding data to the main Table
            df_products = pd.concat([df_products, main_table], ignore_index=True)
            print(df_products.shape)
    
    # Option 3
    elif len(options)==2 and options[0]!=options[1]:
        print(i, 'Two options', options)
        
        # Get page
        browser.get(url)
        attrs_one = browser.find_element_by_xpath('/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[2]/div/div[1]').get_attribute('innerHTML')
        num_tags_one = len(re.findall('a-tag m-variants__attribute-tag', attrs_one))

        attrs_two = browser.find_element_by_xpath('/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[2]/div/div[2]').get_attribute('innerHTML')
        num_tags_two = len(re.findall('a-tag m-variants__attribute-tag', attrs_two))
        
        # Looping through options
        for par_one in range(2,2+num_tags_one):
            sleep(5)

            try:
                print('i', i)
                path_one = '/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[2]/div/div[1]/div[{}]/label'.format(par_one)
                browser.find_element_by_xpath(path_one).click()

                actions = ActionChains(browser)
                for _ in range(15):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                    sleep(0.1)
                sleep(0.2)
                for _ in range(10):
                    actions.send_keys(Keys.PAGE_UP).perform()
                    sleep(0.1)
            except:
                pass


            for par_two in range(2,2+num_tags_two):
                try:
                    sleep(5)
                    path_two = '/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[2]/div/div[2]/div[{}]/label'.format(par_two)
                    browser.find_element_by_xpath(path_two).click()
                    actions = ActionChains(browser)
                    for _ in range(15):
                        actions.send_keys(Keys.PAGE_DOWN).perform()
                        sleep(0.1)

                    sleep(0.2)
                    for _ in range(10):
                        actions.send_keys(Keys.PAGE_UP).perform()
                        sleep(0.1)

                    sleep(5)
                    main_table = get_page_date(url)
                    #display(main_table)
            
                    # Adding data to the main Table
                    df_products = pd.concat([df_products, main_table], ignore_index=True)
                    print(df_products.shape)

                except:
                    pass
    
    
    # Option 4
    elif len(options)==4 and options[0]==options[2] and options[1]==options[3]:
        print(i, 'Two options with dropdown', options)
        arrow_path ='//*[@class="a-icon a-icon--arrow-down m-variants__selection-icon m-variants__selection-icon--closed"]'
        
        # Get page
        browser.get(url)
        attrs_one = browser.find_element_by_xpath('/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[1]').get_attribute('innerHTML')
        num_tags_one = len(re.findall('a-tag m-variants__attribute-tag', attrs_one))

        attrs_two = browser.find_element_by_xpath('/html/body/div[1]/div/div/section[1]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[2]').get_attribute('innerHTML')
        num_tags_two = len(re.findall('a-tag m-variants__attribute-tag', attrs_two))
        
        # Looping through options
        for par_one in range(2,2+num_tags_one):
            try:
                browser.find_element_by_xpath(arrow_path).click()
                sleep(5)
            except:
                pass

            try:
                path_one = '/html/body/div[1]/div/div[2]/section[1]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[1]/div[{}]/label'.format(par_one)
                browser.find_element_by_xpath(path_one).click()
            except:
                pass

            actions = ActionChains(browser)
            for _ in range(15):
                actions.send_keys(Keys.PAGE_DOWN).perform()
                sleep(0.1)

            sleep(4)

            for par_two in range(2,2+num_tags_two):
                try:
                    browser.find_element_by_xpath(arrow_path).click()
                    sleep(5)
                except:
                    pass
                try:
                    path_two = '/html/body/div[1]/div/div[2]/section[1]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[2]/div[{}]/label'.format(par_two)
                    browser.find_element_by_xpath(path_two).click()
                    actions = ActionChains(browser)
                    for _ in range(15):
                        actions.send_keys(Keys.PAGE_DOWN).perform()
                        sleep(0.1)

                    sleep(3)
                    # Get the main page
                    main_table = get_page_date(url)
                    #display(main_table)
                    
                    # Adding data to the main Table
                    df_products = pd.concat([df_products, main_table], ignore_index=True)
                    print(df_products.shape)

                except:
                    pass

In [ ]:
products = df_products.copy()
products.shape

In [ ]:
#with open('Toom-Wood-All-Products.pkl', 'wb') as f:
    #pickle.dump(products, f)

# Processing data

#### Deduplication

In [ ]:
products = products[~products.duplicated(keep='last')]
products.shape

#### Price/Delivery

In [ ]:
test = products.copy()
test.shape

In [ ]:
# Clean the main price
test['Price'] = test['Price'].str.replace(r'[^\,\d]','').str.replace(r'\,','.').astype(float)

In [ ]:
# Delivery and Order Online
test['Order-Delivery-Status'] = test['Delivery-Price'].str.split('\ninkl. 16% MwSt.\n', expand=True)[1]
test['Order-Delivery-Status'] = test['Order-Delivery-Status'].str.replace(r'\n',' ')#

pat = r'Dieses Produkt können wir dir online leider nicht anbieten|Dieser Artikel ist nicht mehr verfügbar im Markt Nordhorn'
test['Order-Delivery-Status'] = test['Order-Delivery-Status'].str.replace(pat,'Not Available Online')

pat = r'Dieses Produkt ist bald wieder verfügbar'
test['Order-Delivery-Status'] = test['Order-Delivery-Status'].str.replace(pat,'Will be available again soon')

pat = r'Bestellbarkeit zur Zeit nicht prüfbar'
test['Order-Delivery-Status'] = test['Order-Delivery-Status'].str.replace(pat,'Delivery Status cannot be checked at the moment')

pat = r'Reservierbar im Markt Nordhorn.+|Im Markt bestellbar in Nordhorn.+'
test['Order-Delivery-Status'] = test['Order-Delivery-Status'].str.replace(pat,'Not Available Online')

In [ ]:
# Other Prices
test['Other-Price'] = test[test['Delivery-Price'].str.contains('Paket')]['Delivery-Price'].str.split('\ninkl. 16% MwSt.\n', expand=True)[0]

#### Delivery Yes/No

In [ ]:
test['Delivery-Status'] = np.nan
for i in test.index:
    if 'Lieferzeit' in test['Order-Delivery-Status'].loc[i]:
        test['Delivery-Status'].loc[i] = 'Yes'
    else:
        test['Delivery-Status'].loc[i] = 'No'

#### SKU

In [ ]:
test['SKU'] = test['SKU'].str.replace(r'\D','')

#### Drop missed columns

In [ ]:
missed_cols = pd.DataFrame((test.isna().sum()/test.shape[0]*100), columns=['Missed'])
cols_del = missed_cols[missed_cols['Missed']>95].index.to_list()
len(cols_del)

In [ ]:
test = test.drop(columns=cols_del)
test.shape

#### Unifying to mm

In [ ]:
test['Breite Descr'] = np.nan
test['Länge Descr'] = np.nan
test['Stärke Descr'] = np.nan

#### Breite

In [ ]:
test['Breite Mult'] = 1
for i in test.index:
    if 'cm' in test['Breite'].astype(str).loc[i]:
        test['Breite Mult'].loc[i] = 10
    
    if 'mm' in test['Breite'].astype(str).loc[i]:
        test['Breite Mult'].loc[i] = 1

In [ ]:
test['Breite Descr'] = test['Breite'].str.replace(r'[a-zA-Z]| ','').str.replace(r'\,','.').astype(float)
test['Breite Descr'] = test['Breite Descr']*test['Breite Mult']

#### Länge

In [ ]:
test['Länge Mult'] = 1
for i in test.index:
    if 'cm' in test['Länge'].astype(str).loc[i]:
        test['Länge Mult'].loc[i] = 10
    
    if 'mm' in test['Länge'].astype(str).loc[i]:
        test['Länge Mult'].loc[i] = 1
        
    if len(re.findall(r' m$', test['Länge'].astype(str).loc[i]))>0:
        test['Länge Mult'].loc[i] = 1000

In [ ]:
test['Länge Descr'] = test['Länge'].str.replace(r'[a-zA-Z]| ','').str.replace(r'\,','.').astype(float)
test['Länge Descr'] = test['Länge Descr']*test['Länge Mult']

#### Stärke

In [ ]:
test['Stärke Mult'] = 1
for i in test.index:
    if 'cm' in test['Stärke'].astype(str).loc[i]:
        test['Stärke Mult'].loc[i] = 10
    
    if 'mm' in test['Stärke'].astype(str).loc[i]:
        test['Stärke Mult'].loc[i] = 1
        
    if len(re.findall(r' m$', test['Stärke'].astype(str).loc[i]))>0:
        test['Stärke Mult'].loc[i] = 1000

In [ ]:
test['Stärke Descr'] = test['Stärke'].str.replace(r'[a-zA-Z]| ','').str.replace(r'\,','.').astype(float)
test['Stärke Descr'] = test['Stärke Descr']*test['Stärke Mult']

#### Dimentions Missed. Header dims

In [ ]:
missed_dims = test[(test['Breite Descr'].isna()) |
                     (test['Länge Descr'].isna()) |
                     (test['Stärke Descr'].isna())].index.to_list()
len(missed_dims)

In [ ]:
test['Header Dims'] = test['Header'].str.replace(r'^.+?(?=\d+ x)|^.+?(?=\d+x)','')

In [ ]:
test['Dims Mult'] = test['Header Dims'].str.extract(r'(cm)|(mm)', expand=True)[0]
test['Dims Mult'] = test['Dims Mult'].fillna(test['Header Dims'].str.extract(r'(cm)|(mm)', expand=True)[1])

test['Dims Mult'] = test['Dims Mult'].str.replace(r'(?<=cm).+|(?<=mm).+','')
test['Dims Mult'] = test['Dims Mult'].str.replace(r'.+(?=mm)|.+(?=cm)','')
test['Dims Mult'] = test['Dims Mult'].str.replace(r' $|^ ','')
test['Dims Mult'] = test['Dims Mult'].str.replace('cm','10').str.replace('mm','1')

test['Dims Mult'] = test['Dims Mult'].apply(lambda x: 1 if x!='1' and x !='10' else x).astype(float)

In [ ]:
#test['Dims Mult'] = test['Dims Mult'].apply(lambda x: np.nan if x!='1' and x!='10' else x).astype(float)

In [ ]:
test['Breite Header'] = np.nan
test['Länge Header'] = np.nan
test['Stärke Header'] = np.nan

for i in missed_dims:
    width = np.nan
    thick = np.nan
    length = np.nan
    
    try:
        length = test['Header Dims'].loc[i].split('x')[0]
    except:
        pass
    try:
        width = test['Header Dims'].loc[i].split('x')[1]
    except:
        pass
    try:
        thick = test['Header Dims'].loc[i].split('x')[2]
    except:
        pass     
    
    test['Breite Header'].loc[i] = width
    test['Länge Header'].loc[i] = length
    test['Stärke Header'].loc[i] = thick

In [ ]:
for col in ['Breite Header','Länge Header','Stärke Header']:
    
    test[col] = test[col].fillna('').astype(str).str.replace(r'[a-zA-Z]| ','').str.replace(r'\,','.').str.replace(r' ','')#
    test[col] = test[col].fillna('').astype(str).str.replace(r'[^\.\d]|\.$','').str.replace(r'^\.','')
    test[col] = test[col].apply(lambda x: np.nan if x==' ' or x=='' else x )
    
    test[col] = test[col].astype(float)

In [ ]:
for i in missed_dims:
    for col in ['Breite Header','Länge Header','Stärke Header']:
        test[col].loc[i] = test[col].loc[i]*test['Dims Mult'].loc[i]

In [ ]:
missed_dims = test[(test['Breite Descr'].isna()) &
                   (test['Länge Descr'].isna()) &
                   (test['Stärke Descr'].isna()) &
                   (test['Breite Header'].isna()) &
                   (test['Länge Header'].isna()) &
                   (test['Stärke Header'].isna())].index.to_list()
len(missed_dims)

In [ ]:
test['Breite Header'] = test['Breite Header'].fillna(test['Breite Descr'])
test['Länge Header'] = test['Länge Header'].fillna(test['Länge Descr']) 
test['Stärke Header'] = test['Stärke Header'] .fillna(test['Stärke Descr']) 


In [ ]:
dims_cols = ['Header','Header Dims','Breite New','Länge New','Stärke New','Dims Mult',
             'Breite Header','Länge Header','Stärke Header']

In [ ]:
test[['Header','Header Dims','Breite Descr','Länge Descr','Stärke Descr','Dims Mult',
     'Breite Header','Länge Header','Stärke Header']].loc[missed_dims].sample()

## Main parameters

In [ ]:
test['Header Meta'] = test['Header'].str.replace(r'[^a-zA-Z]', ' ').str.replace(r'  +', ' ').str.replace(r' x .+', '')
test['Header Meta'] = test['Header Meta'].str.replace(r' wei|toom ', '').str.replace(r' [A-Z]$|^[A-Z] | [a-z]$|^ [a-z]', '')
test['Header Meta'] = test['Header Meta'].str.replace(r' [a-z] | [A-Z] ', ' ').str.replace(r'^ |easy| St ck|WandArt| mm| cm', '')#.value_counts() #.value_counts()#

In [ ]:
to_del =['Marke','Räume','Serie','Höhe', 'Einsatzbereich', 'Lieferumfang','Ausführung',
         'Inhalt','Geeignet für','Kanten','Kantenausführung','Maserung','Modell','Zuschnitts-Option']

### Wood Types

In [ ]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [ ]:
test['Header Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Header Meta'].loc[i].lower()))>0:
        #if len(wood.lower() in test['Header Meta'].loc[i].lower():
            test['Header Wood'].loc[i] = wood

In [ ]:
test['Description Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Description'].loc[i].lower()))>0:
            test['Description Wood'].loc[i] = wood

In [ ]:
for col in ['Header Wood','Description Wood']:
    test[col] = test[col].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test[['Header','Header Wood',
      'Description','Description Wood']].isna().sum()/test.shape[0]*100

In [ ]:
test[['Header','Header Wood',
      'Description','Description Wood']].sample(3)

#### Combining columns

In [ ]:
test['Wood Type'] = test['Materialspezifizierung'].fillna(test['Material']).fillna(test['Header Wood']).fillna(test['Description Wood'])

### Cheking wrong typs and woods

In [ ]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Stahl', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
             'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Spanholz', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Holz, Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Kirschbaum','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Holz, Kunststoff','Lackiert','WPC','Kunststoff', 
              'Stahl','Weich-PVC','Kunststoff, Metall','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
             'Kunststoff, Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte']

In [ ]:
test[(~test['Wood Type'].isin(wrong_wood))].shape

In [ ]:
test = test[(~test['Wood Type'].isin(wrong_wood))]
test.shape

In [ ]:
test['Wood Type'] = test['Wood Type'].str.lower()

# Combine all columns to mine text 

In [ ]:
comb_cols = ['Materialspezifizierung',  'Paketinhalt', 
           'Header', 'Description', 
           'Herstellerfarbe', 'Artikeltyp', 'Oberfläche', 'Marke', 'Farbe',
           'Hinweistext', 'Tiefe', 'Ausführung', 'Tragkraft pro Boden',
           'Verdeckte Montageleiste', 'Gewicht', 'Inhalt', 'Einsatzbereich',
           'Material', 'Serie', 'Montageart', 'Lieferumfang', 'Höhe', 'Tragkraft',
           'Durchmesser Rad', 'Design', 'Holzart',
           'Herkunftsland', 'Kantenverarbeitung', 'Geeignet für',
           'Nutwangenstärke', 'Lichtdurchlässig', 'Set', 'Maserung', 'Modell',
           'Kantenausführung', 'Zuschnitts-Option', 'Durchmesser', 'Form',
           'Sortierung', 'Räume', 'Kanten', 'Norm', 'Zuschnittartikel',
           'Passend für', 'Länge min.', 'Kabelführung', 'Endlos verlegbar',
           'Packungsinhalt reicht für', 'Fugenart', 'Deckfläche', 'Profil',
           'Geeignet für Anstriche', 'Haltbarkeit im Außenbereich', 'Harzhaltig',
           'Verzug', 'Schwind- & Quellverhalten', 'Rissig', 'Bearbeitbarkeit',
           'Art der Verlegung', 'Eigenschaften', 'Nutmaß',
           'Geeignet für Feuchtraum', 'Türvariante', 'Order-Delivery-Status']

In [ ]:
test['Combined'] = ''
for col in comb_cols:
    test['Combined'] = test['Combined'] + '; ' + test[col].astype(str)

In [ ]:
test['Combined'] = test['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

### Surface

In [ ]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [ ]:
test['Header Surface Treat'] = ''
for i in test.index:
    test['Header Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Header Meta'].loc[i].lower()))>0:
            test['Header Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Header Meta'].loc[i].lower():
            test['Header Surface Treat'].loc[i].append(surface)
    try:
        test['Header Surface Treat'].loc[i] = ', '.join(test['Header Surface Treat'].loc[i])
    except:
        pass

In [ ]:
test['Description Surface Treat'] = ''
for i in test.index:
    test['Description Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Description'].loc[i].lower()))>0:
            test['Description Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Description'].loc[i].lower():
            test['Description Surface Treat'].loc[i].append(surface)
    try:
        test['Description Surface Treat'].loc[i] = ', '.join(test['Description Surface Treat'].loc[i])
    except:
        pass

In [ ]:
test['Combined Surface Treat'] = ''
for i in test.index:
    test['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Combined'].loc[i].lower() and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    try:
        test['Combined Surface Treat'].loc[i] = ', '.join(test['Combined Surface Treat'].loc[i])
    except:
        pass

In [ ]:
for col in ['Combined Surface Treat', 'Header Surface Treat','Description Surface Treat']:
    test[col] = test[col].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test[['Header','Header Surface Treat',
      'Description','Description Surface Treat',
      'Add Description','Combined Surface Treat']].isna().sum()/test.shape[0]*100

In [ ]:
cols = ['Header','Header Surface Treat',
        'Description','Description Surface Treat',
        'Oberfläche','Combined Surface Treat']

In [ ]:
test[((~test['Header Surface Treat'].isna()) |
     (~test['Description Surface Treat'].isna())) & 
     (test['Combined Surface Treat'].isna()) ][cols].shape[0]

In [ ]:
#test['Combined Surface Treat'].value_counts()

In [ ]:
test[((~test['Header Surface Treat'].isna()) |
     (~test['Description Surface Treat'].isna())) &
     (test['Oberfläche'].isna()) ][cols].sample(3)

### Drying method

In [ ]:
dry_method = ['technischer trocknung','technisch getrocknet', 'kammer getrocknet',
               'kammertrocken', 'AD (luftgetrocknet)',
               'KD (Künstlich getrocknet)']

In [ ]:
test['Combined Drying Method'] = ''
for i in test.index:
    test['Combined Drying Method'].loc[i] = []
    for method in dry_method:
        if method.lower() in test['Combined'].loc[i].lower() and method.lower() not in ', '.join(test['Combined Drying Method'].loc[i]):
            test['Combined Drying Method'].loc[i].append(method)
        
    try:
        test['Combined Drying Method'].loc[i] = ', '.join(test['Combined Drying Method'].loc[i])
    except:
        pass

In [ ]:
test['Combined Drying Method'] =  test['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [ ]:
test['Combined Drying Method'].value_counts()

### Surface Quality

In [ ]:
keywords.head(1)

In [ ]:
surf_quality = ['NSI', 'SI']

In [ ]:
test['Combined Surface Quality'] = ''
for i in test.index:
    test['Combined Surface Quality'].loc[i] = []
    for surf_qual in surf_quality:
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(test['Combined Surface Quality'].loc[i]):
            test['Combined Surface Quality'].loc[i].append(surf_qual)
            
    try:
        test['Combined Surface Quality'].loc[i] = ', '.join(test['Combined Surface Quality'].loc[i])
    except:
        pass
test['Combined Surface Quality'] =  test['Combined Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )   

In [ ]:
test['Combined Surface Quality'].value_counts()

### Surface Quality

In [ ]:
quality = ['A-Sortierung', 'B-Sortierung', 'C-Sortierung', 'A/B', 'B/C','Kl. A', '1.Wahl Sortierung', 'Altholz']

In [ ]:
test['Combined Quality'] = ''
for i in test.index:
    test['Combined Quality'].loc[i] = []
    for qual in quality:
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(test['Combined Quality'].loc[i]):
            test['Combined Quality'].loc[i].append(qual)
            
    try:
        test['Combined Quality'].loc[i] = ', '.join(test['Combined Quality'].loc[i])
    except:
        pass
test['Combined Quality'] =  test['Combined Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Quality'].value_counts()

### Sorting Class

In [ ]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe Tragfähigkeit', 'mittlere Tragfähigkeit',
               'hohe Tragfähigkeit']

In [ ]:
test['Combined Sorting Class'] = ''
for i in test.index:
    test['Combined Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(test['Combined Sorting Class'].loc[i]):
            test['Combined Sorting Class'].loc[i].append(sort_class)
       
    try:
        test['Combined Sorting Class'].loc[i] = ', '.join(test['Combined Sorting Class'].loc[i])
    except:
        pass
test['Combined Sorting Class'] =  test['Combined Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Sorting Class'].value_counts()

#### Combining columns

In [ ]:
test['Surfase Treatment'] = test['Oberfläche'].fillna(test['Description Surface Treat']).fillna(test['Header Surface Treat'])
test['Surfase Treatment'] = test['Surfase Treatment'].str.lower()

### Combinig columns for parameters

In [ ]:
test['Product Type'] = test['Artikeltyp'].fillna(test['Header Meta'])
test['Color'] = test['Herstellerfarbe'].fillna(test['Farbe'])

In [ ]:
for i in test.index:
    for wood in woods:
        if wood.lower() in test['Product Type'].loc[i].lower():
            test['Product Type'].loc[i] = re.sub(wood,'', test['Product Type'].loc[i])

In [ ]:
test['Product Type'] = test['Product Type'].str.replace(r' +', ' ')

### Cleaning NaNs

In [ ]:
for col in test.columns:
    test[col] =  test[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' else x)

### Product type

In [ ]:
test['Product Type New'] = test['Product Type'].copy()

In [ ]:
for i in test.index:
    
    if 'regalbo' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'regalboden'
        
    if 'holzrundst' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'holzrundstab'
        
    if 'bauplatte' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'bauplatte'
        
    if 'leiste' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'leiste'  
    
    if 'profil' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'profil' 
    
    if 'woodstone' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'woodstone'
    
    if 'rahmen' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'rahmenholz'
    
    if 'allwetter' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'allwetterholz'
    
    if 'leimholz' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'leimholz'
    
    if 'konstruktion' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'konstruktionsholz'
    
    if 'arbeitsplatte' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'arbeitsplatte'
    
    if 'lamellent' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'lamellent'
    
    if 'terrassendiel' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'terrassendiele'
    
    if 'stab' in  test['Product Type New'].astype(str).loc[i].lower():
        test['Product Type New'].loc[i] = 'stab'
    
  
test['Product Type New'].nunique()

In [ ]:
right_types = test['Product Type New'].value_counts()[:35].index.to_list()
#test = test[test['Product Type New'].isin(right_types)]
test[test['Product Type New'].isin(right_types)].shape

In [ ]:
test[test['Product Type New'].isin(right_types)].shape

In [ ]:
test = test[(test['Product Type New']!='praktikus kantenumleimer ') & 
            (test['Product Type New']!='verlegeplatte span geschliffen') & 
            (test['Product Type New']!='belfront room ') & 
            (test['Product Type New']!='verlegeplatte span geschliffen') & 
            (test['Product Type New']!='viertelstab nadelholz') &
            (test['Product Type New']!='steckpaneele vario edelwei') & 
            (test['Product Type New']!='regalelement segment')& 
            (test['Product Type New']!='endstück')&
            (test['Product Type New']!='holzkugel')
           ]
test.shape

In [ ]:
test['Product Type New'] = test['Product Type New'].str.lower()

### Keywords

In [ ]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [ ]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [ ]:
keywords.head(1)

In [ ]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' else x)

In [ ]:
#keywords['Wood Type'].loc[31] = 'Holz'

### Matching keywords

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
matched_df = test.copy()

In [ ]:
matched_df = matched_df.drop(columns='Product Type')

In [ ]:
ren_cols = {'Product Type New':'Product Type',
           'Combined Sorting Class':'Sorting Class',
           'Combined Quality':'Quality',
            'Combined Drying Method':'Drying Method',
            'Combined Surface Treat':'Surface Treatment',
           'Combined Surface Quality':'Surface Quality'}

In [ ]:
matched_df = matched_df.rename(columns=ren_cols)

In [ ]:
main_cols = ['URL','SKU','Header','Product Type','Wood Type','Surface Treatment','Surface Quality',
             'Drying Method','Sorting Class','Quality']

In [ ]:
matched_df[main_cols].head(1)

In [ ]:
matched_df['Matched Keywords'] = np.nan
matched_df['Other Keywords'] = np.nan
matched_df = matched_df.fillna('-777')
keywords = keywords.fillna('-999')

In [ ]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
matched_df = matched_df.apply(get_keywords, axis=1)

In [ ]:
matched_df[['Matched Keywords','Other Keywords']].isna().sum()/matched_df.shape[0]*100, matched_df.shape

### Cleaning Product Types

In [ ]:
matched_df[(~matched_df['Matched Keywords'].isna()) &
           (~matched_df['Wood Type'].isin(wrong_wood_second))
          ]['Product Type'].shape#.nunique()#value_counts()

#### cleaning product types

In [ ]:
for i in matched_df.index:
    
    for word in ['kantenumleimer','bodendiele','umleimer','holzfliese','rauspund','hobeldiele',
                'vielzweckplatte','grobspanplatte','brettschichtholz','universalbrett',
                'unterkonstonstruktion','wandabschluss zubeh','dekorkante getalit','massivholzplatte',
                'verbindungsecken','verlegeplatte']:
        
        if word in matched_df['Product Type'].astype(str).loc[i].lower():
            matched_df['Product Type'].loc[i] = word
  
matched_df[(~matched_df['Matched Keywords'].isna()) &
           (~matched_df['Wood Type'].isin(wrong_wood_second))
          ]['Product Type'].nunique()

In [ ]:
matched_df['Product Type'] = matched_df['Product Type'].str.replace('uk latten','uk latte')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('latte impr gniert','latte')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('ausgleichsh lzer rotbuche','ausgleichshölzer')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('ckwandsystemset fantasie|ckwand system set getalit edelstahl optik tlg ','rückwandsystemset')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('konuspl ttchen ','konusplättchen ')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('klenk holz','')
matched_df['Product Type'] = matched_df['Product Type'].str.replace('st lpschalungsbretter ','stülpschalungsbretter')
matched_df['Product Type'] = matched_df['Product Type'].str.replace(r'^ | $','')
matched_df['Product Type'] = matched_df['Product Type'].str.replace(r'diele grau impr gniert geriffelt glatt','diele')
matched_df['Product Type'] = matched_df['Product Type'].str.replace(r'wpc klickfliesen er set anthrazit','klickfliesen')
matched_df['Product Type'] = matched_df['Product Type'].str.replace(r'st lpschalung','stülpschalung')

In [ ]:
matched_df['Product Type'] = matched_df['Product Type'].str.lower()

In [ ]:
matched_df[(~matched_df['Matched Keywords'].isna()) &
           (~matched_df['Wood Type'].isin(wrong_wood_second))
          ]['Product Type'].value_counts()

In [ ]:
wrong_wood_second = ['Polycarbonat', 'Kunststoff', 'PVC', 'German Compact Composite (GCC)', 
                     'Foliert', 'WPC', 'Metall', 'Multiplex', 'Beschichtet', 'Stahl']
#wrong_wood_second = [x for x in matched_df[(~matched_df['Matched Keywords'].isna())]['Wood Type'].unique() if  x in wrong_wood]

In [ ]:
matched_df[(~matched_df['Matched Keywords'].isna()) &
           (~matched_df['Wood Type'].isin(wrong_wood_second))
          ].shape

In [ ]:
matched_df[(matched_df['Matched Keywords'].isna()) & 
          (matched_df['Wood Type']=='Holz')][main_cols].shape#['Wood Type'].value_counts()#.shape

In [ ]:
matched_df[(matched_df['SKU'].str.contains('7200058')) &
            (~matched_df['Matched Keywords'].isna()) &
            (~matched_df['Wood Type'].isin(wrong_wood))]['Product Type'].shape#value_counts()

In [ ]:
matched_df[(~matched_df['Matched Keywords'].isna()) #&
           #(~matched_df['Wood Type'].isin(wrong_wood))
          ]['Product Type'].nunique()#value_counts()

### Pre-cleaning

In [ ]:
for col in matched_df.columns:
    matched_df[col] = matched_df[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [ ]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
#sheets-juputer@jupyter-sheets-270906.iam.gserviceaccount.com

In [ ]:
cols = ['Website Name', 'URL','Header','Product Type','Wood Type', 'Breite Header', 'Länge Header', 'Stärke Header', 
        'Price','Delivery-Status','Quality', 'Surface Quality','Surface Treatment','Drying Method',
        'Sorting Class','SKU','Order-Delivery-Status' ]

In [ ]:
sheets_clean = matched_df[(~matched_df['Matched Keywords'].isna()) &
                          (~matched_df['Wood Type'].isin(wrong_wood_second)) ]
sheets_clean.shape

In [ ]:
sheets_clean = sheets_clean[(~sheets_clean.duplicated(subset='URL', keep='last'))]
sheets_clean.shape

In [ ]:
sheets_clean['Website Name'] = 'www.toom.de'

In [ ]:
missed_dims = sheets_clean[(sheets_clean['Breite Header'].isna()) &
                           (sheets_clean['Länge Header'].isna()) &
                           (sheets_clean['Stärke Header'].isna())].index.to_list()
len(missed_dims)

In [ ]:
#sheets_clean[cols].loc[missed_dims]

In [ ]:
ren_cols = {'Header':'Product Title',
           'Breite Header':'Width (mm)',
           'Länge Header':'Length (mm)', 
           'Stärke Header':'Thickness (mm)',
            'Price':'Price (Euro)',
            'Order-Delivery-Status':'Delivery Status',
            'Delivery-Status':'Order Delivery (Y/N)'}

In [ ]:
sheets_clean = sheets_clean[cols].rename(columns = ren_cols).fillna('-')
sheets_clean.head(1)

In [ ]:
d2g.upload(sheets_clean,
           spreadsheet_key,
           'Toom Clean Data',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

### Raw Data Toom

In [ ]:
sheets_raw = matched_df[(~matched_df.duplicated(subset='URL', keep='last'))]
sheets_raw.shape

In [ ]:
sheets_raw['Website Name'] = 'www.toom.de'

In [ ]:
sheets_raw = sheets_raw[cols].rename(columns = ren_cols).fillna('-')
sheets_raw.head(1)

In [ ]:
d2g.upload(sheets_raw,
           spreadsheet_key,
           'Toom Raw Data',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)